In [2]:
import numpy as np
import polars as pl

In [10]:
df = pl.DataFrame({
    'foo': [1, 2, 3, 2, 1],
    'bar': [1, 0, 1, 1, 0]})
df

foo,bar
i64,i64
1,1
2,0
3,1
2,1
1,0


In [11]:
df.select([
    pl.col('foo').sort().head(2),
    pl.col('bar').filter(pl.col('foo') == 1)])

foo,bar
i64,i64
1,1
1,0


In [22]:
df = pl.DataFrame({
    'nrs': [1, 2, 3, None, 5, 6],
    'names': ['foo', 'ham', 'spam', 'egg', None, 'spam'],
    'random': np.random.rand(6),
    'group': list('AABCBB')})
df

nrs,names,random,group
i64,str,f64,str
1,"""foo""",0.098545,"""A"""
2,"""ham""",0.03964,"""A"""
3,"""spam""",0.061895,"""B"""
null,"""egg""",0.038327,"""C"""
5,null,0.41853,"""B"""
6,"""spam""",0.972987,"""B"""


In [23]:
df.select([
    pl.col('names').n_unique().alias('n_names1'),
    pl.col('names').unique().count().alias('n_names2')])

n_names1,n_names2
u32,u32
5,5


In [24]:
df.select([
    pl.sum('random').alias('sum'),
    pl.min('random').alias('min'),
    pl.max('random').alias('max'),
    pl.col('random').max().alias('maxagain'),
    pl.std('random').alias('sd'),
    pl.var('random').alias('var')])

sum,min,max,maxagain,sd,var
f64,f64,f64,f64,f64,f64
1.629925,0.038327,0.972987,0.972987,0.373009,0.139136


In [25]:
df.select([
    pl
    .col('names')
    .filter(pl.col('names').str.contains(r'am$'))
    .count()
    .alias('n_names_ending_in_AM')])

n_names_ending_in_AM
u32
3


In [26]:
df.select([
    pl
    .when(pl.col('random') > 0.5)
    .then(1)
    .otherwise(pl.col('random')) 
    * pl.sum('nrs')])

literal
f64
1.67527
0.67388
1.052218
0.65156
7.115011
17.0


In [27]:
df.select([
    pl.col('*'),
    pl.col('random').sum().over('group').alias('∑random/groups'),
    pl.col('random').list().over('names').alias('random/name')])

nrs,names,random,group,∑random/groups,random/name
i64,str,f64,str,f64,list[f64]
1,"""foo""",0.098545,"""A""",0.138185,[0.098545]
2,"""ham""",0.03964,"""A""",0.138185,[0.03964]
3,"""spam""",0.061895,"""B""",1.453412,"[0.061895, 0.972987]"
null,"""egg""",0.038327,"""C""",0.038327,[0.038327]
5,null,0.41853,"""B""",1.453412,[0.41853]
6,"""spam""",0.972987,"""B""",1.453412,"[0.061895, 0.972987]"
